In [ ]:
!pip install -q transformers datasets scikit-learn accelerate


In [ ]:
import numpy as np
print(f"NumPy version: {np.__version__}")  # Should show 1.26.4

NumPy version: 2.0.2


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
import torch
import numpy as np
import pandas as pd

In [ ]:
model_name = "l3cube-pune/marathi-bert-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: pos/neg/neutral


tokenizer_config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marathi-bert-v2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_ST/MahaSent_ST_Train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_ST/MahaSent_ST_Val.csv")
test_df = pd.read_csv('/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_ST/MahaSent_ST_Test.csv')
train_df.head()
# val_df.head()
# test_df.head()

,Unnamed: 0,marathi_text,label
0,0,मी तो मूर्ख आहे.,-1
1,1,आईने आत्महत्या केली आणि माझे वडील पळून गेले.,-1
2,2,- त्यांनी फक्त जॉयला मारले?,-1
3,3,मला माफ करा मी आज तिथेच केले नाही.,-1
4,4,"नाही, नक्कीच नाही.",-1


In [ ]:
train_df["label"] += 1
val_df["label"] += 1
test_df["label"] += 1
train_df.head()
train_df.dropna(subset=["marathi_text", "label"], inplace=True)
val_df.dropna(subset=["marathi_text", "label"], inplace=True)
test_df.dropna(subset=["marathi_text", "label"], inplace=True)
print(train_df.isnull().sum())

Unnamed: 0      0
marathi_text    0
label           0
dtype: int64


In [ ]:
# Step 3: Convert to Hugging Face Dataset format
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
#  Step 4: Tokenize the tweets
def tokenize_function(batch):
    return tokenizer(batch["marathi_text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
# Format for PyTorch
columns = ['input_ids', 'attention_mask', 'label']
train_dataset.set_format(type='torch', columns=columns)
val_dataset.set_format(type='torch', columns=columns)
test_dataset.set_format(type='torch', columns=columns)

In [ ]:

#  Initialize Trainer

training_args = TrainingArguments(
    output_dir="./results",                   # 📁 Where to save checkpoints
    eval_strategy="epoch",              # 🧪 Evaluate after each epoch
    save_strategy="epoch",                    # 💾 Save model after each epoch
    per_device_train_batch_size=8,            # 🧠 Reduce memory usage
    per_device_eval_batch_size=8,             # ⚖️ Matching batch size for eval
    num_train_epochs=2,                       # 🔁 You can try 1 or 2 for speed
    learning_rate=2e-5,                       # 🚀 Good default for BERT
    warmup_steps=100,                         # 🔥 Warmup helps training stability
    weight_decay=0.01,                        # 📉 Helps avoid overfitting
    logging_dir="./logs",                     # 📂 Logs directory
    logging_steps=50,                         # 📝 Log every 50 steps
    load_best_model_at_end=True,              # 🏆 Automatically load best model
    metric_for_best_model="accuracy",         # 🎯 Use accuracy to pick best
    save_total_limit=1,                       # 💾 Keep only last best model
    fp16=True,                                # ⚡ Mixed precision for faster training
    report_to="none",                         # 📊 Turn off W&B if you’re not using it
)


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-485065071.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Step 5: Train!
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.624400,0.529196,0.808000
2,0.480400,0.514254,0.820667


TrainOutput(global_step=3000, training_loss=0.6007859382629395, metrics={'train_runtime': 452.14, 'train_samples_per_second': 53.081, 'train_steps_per_second': 6.635, 'total_flos': 1578680506368000.0, 'train_loss': 0.6007859382629395, 'epoch': 2.0})

In [ ]:
# Run evaluation on the TEST dataset
test_results = trainer.evaluate(eval_dataset=test_dataset)
print(test_results)

{'eval_loss': 0.5627264976501465, 'eval_accuracy': 0.796, 'eval_runtime': 6.7086, 'eval_samples_per_second': 223.594, 'eval_steps_per_second': 28.024, 'epoch': 2.0}


In [ ]:
# After training
val_metrics = trainer.evaluate()          # still okay to see validation
print("Validation metrics:", val_metrics)

test_metrics = trainer.evaluate(test_dataset)   # ✅ this matches the paper
print("Test metrics:", test_metrics)

Validation metrics: {'eval_loss': 0.5142541527748108, 'eval_accuracy': 0.8206666666666667, 'eval_runtime': 6.539, 'eval_samples_per_second': 229.391, 'eval_steps_per_second': 28.75, 'epoch': 2.0}
Test metrics: {'eval_loss': 0.5627264976501465, 'eval_accuracy': 0.796, 'eval_runtime': 7.9058, 'eval_samples_per_second': 189.734, 'eval_steps_per_second': 23.78, 'epoch': 2.0}
